# Задание 1
1. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [4]:
import pandas as pd

In [7]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [9]:
ratings_grouped = ratings.groupby('userId').count().reset_index()
ratings_filtered = ratings_grouped[ratings_grouped['rating']>=100]
film_fans_user_ids = ratings_filtered['userId'].tolist()
ratings_100 = ratings[ ratings['userId'].isin(film_fans_user_ids) ]
ratings_100.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [106]:
ratings_100gp = ratings_100.groupby('userId').agg(['min', 'max'])['timestamp']
ratings_100gp['delta'] = ratings_100gp['max'] - ratings_100gp['min']
LTV = ratings_100gp['delta'].mean()
#поскольку всё в секундах, спокойно переводим в человеческий формат :)
print(f'Среднее время жизни пользователей, выставивших больше 100 оценок составляет {round(LTV/60/60/24/30, 1)} месяцев')

Среднее время жизни пользователей, выставивших больше 100 оценок составляет 15.2 месяцев


# Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [ ]:
#Задание какое-то подозрительно элементарное в решении. Я что-то не понял в условии?

In [76]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [77]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [78]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [89]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [100]:
client_base_cut = pd.DataFrame(client_base['client_id'])
client_base_cut = (client_base_cut.merge(rzd, how = 'left', on = 'client_id')
                .merge(auto, how = 'left', on = 'client_id')
                .merge(air, how = 'left', on = 'client_id'))
client_base_cut

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [101]:
client_base_full = (client_base.merge(rzd, how = 'left', on = 'client_id')
                .merge(auto, how = 'left', on = 'client_id')
                .merge(air, how = 'left', on = 'client_id'))
client_base_full

,address,client_id,rzd_revenue,auto_revenue,air_revenue
0,Комсомольская 4,111,1093.0,NaN,NaN
1,Энтузиастов 8а,112,2810.0,NaN,NaN
2,Левобережная 1а,113,10283.0,57483.0,NaN
3,Мира 14,114,5774.0,83.0,NaN
4,ЗЖБИиДК 1,115,981.0,912.0,81.0
5,Строителей 18,116,NaN,4834.0,4.0
6,Панфиловская 33,117,NaN,98.0,13.0
7,Мастеркова 4,118,NaN,NaN,173.0


# Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

1.Если представить что у нас интернет-магазин с доставкой (жд, авто, авиа) и что по каждому пользователю (user_id) есть несколько строк с широтой и долготой его обращения в приложение для совершения покупки, то имея эти данные мы можем:
   - рассчитать расстояние по широте/долготе пользователя и широте/долготе расположения нашего склада, с которого отправится заказ. Для этого используем тригонометрическую формулу для окружности сферы:
    
\begin{equation*}
cos(d) = sin(φА)·sin(φB) + cos(φА)·cos(φB)·cos(λА − λB) \\
\end{equation*}
               
                где φА и φB — широты, λА, λB — долготы данных пунктов, d — расстояние между пунктами, 
                измеряемое в радианах длиной дуги большого круга земного шара. 
                Расстояние между пунктами, измеряемое в километрах, определяется по формуле:
                
\begin{equation*}
                L = d·R
\end{equation*}

                где R = 6371 км — средний радиус земного шара.
                
   - полученное расстояние агрегируем по каждому пользователю и каждому виду транспортировки (жд, авто, авиа), в качестве меры используем медиану.
   - практический выхлоп - имея способ доставки и медиану удалению клиента от места расположения товара, потенциально понимаем вероятную скорость доставки.

2.Другой вариант, более простой, исходить из предположения, что нам важно знать откуда чаще всего выходит в сеть для заказа покупки пользователь, в этом случае просто берём моду широты и моду долготы.

3.Продолжение идеи п.2. Может быть такая ситуация, что места выхода несколько колеблятся (вдруг у нас такая точная аппаратура), тогда моды как таковой не будет по пользователю. Тогда, с помощью косинусного расстояния векторов находим самые близкие друг к другу широту и долготу, допустим на уровне сходиомости (0,95 или выше, надо смотреть на данных), считаем среднее для них (ведь таких комбинаций по пользователю тоже может быть несколько) и уже с этого берём моду. 
Практический выхлоп подхода 2 и 3 такой же, понимаем, где чаще находится клиент, можем ему какой-нибудь наш магазин по соседству таргетировано кинуть рекламой :)